03-JUL-18

Adaptation of "LIME - basic LR"

Trains on the hyperpartisan data

Tests on annotated data from Briefr

In [1]:
import sys
sys.path.insert(0,'/Users/rick/factmata/factnlp-experimental/lime')

# sys.path

In [2]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function
import pandas as pd
import sys
import re
import pandas as pd
from IPython.display import display
from datetime import datetime
import datetime
from nltk.corpus import stopwords
from collections import Counter


import time


/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
import sys
sys.path.append('/Users/rick/factmata/factmata-quality-engine/factnlp')
sys.path.append('/Users/rick/factmata/utils')
sys.path.append('/Users/rick/factmata/fastText')
# sys.path

In [4]:
from factnlp.category.category_predictor import CategoryPredictor

{"level": "INFO", "message": "'pattern' package not found; tag filters are not available for English", "log_timestamp": "2018-09-07 09:20:14.696441", "filename": "textcleaner.py", "function": "<module>", "path": "/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/site-packages/gensim/summarization/textcleaner.py", "logger_name": "summarizer.preprocessing.cleaner", "lineno": 37, "process": 46898, "thread": 140736053298048, "details": {}}


In [5]:
import settings
import distributor.tasks
models = distributor.tasks.models_loader_factory()

/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


{"level": "INFO", "message": "Loading models", "log_timestamp": "2018-09-07 09:20:17.201393", "filename": "tasks.py", "function": "models_loader_factory", "path": "/Users/rick/factmata/factmata-quality-engine/factnlp/distributor/tasks.py", "logger_name": "distributor.tasks", "lineno": 53, "process": 46898, "thread": 140736053298048, "details": {"message": "Loading models"}}
{"level": "INFO", "message": "Loading models", "log_timestamp": "2018-09-07 09:20:17.204148", "filename": "tasks.py", "function": "models_loader_factory", "path": "/Users/rick/factmata/factmata-quality-engine/factnlp/distributor/tasks.py", "logger_name": "distributor.tasks", "lineno": 53, "process": 46898, "thread": 140736053298048, "details": {"message": "Loading models"}}


# Input parameters

In [6]:
# Google drive paths

labelled_data_path = '/hyperpartisanship/CF labelled data'
training_data_path = '/Users/mariarmestre/Projects/factnlp/models/hyperpartisanship/current/'

output_data_path = '/hyperpartisanship/error analysis/with CF labelled data'

# Load data & model

In [7]:
## Production model

production_hp = models.get_hyperpartisanship_model()
production_hp

{"level": "INFO", "message": "Loading Politics Classifier Model", "log_timestamp": "2018-09-07 09:20:17.217199", "filename": "modelsloader.py", "function": "__load_hyperpartisanship_model", "path": "/Users/rick/factmata/factmata-quality-engine/factnlp/factnlp/modelsloader.py", "logger_name": "factnlp", "lineno": 72, "process": 46898, "thread": 140736053298048, "details": {}}
{"level": "INFO", "message": "Loading Hyperpartisanship Model", "log_timestamp": "2018-09-07 09:21:02.288272", "filename": "modelsloader.py", "function": "__load_hyperpartisanship_model", "path": "/Users/rick/factmata/factmata-quality-engine/factnlp/factnlp/modelsloader.py", "logger_name": "factnlp", "lineno": 74, "process": 46898, "thread": 140736053298048, "details": {}}


## Wrapper function to provide predict_proba() for LIME

In [8]:
# ad hoc func to serve as predict_proba() for LIME
import json
classes = ['Biased','Unbiased']

def fm_predict_proba(in_text_list,verbose=False):
  """
  Takes a list of texts and for each gets a probability of being hyperpartisan from the hyperpartisan model
  Then extends the probability returned into a tuple of (probability of true, probability of false) as required
  by the LIME explain.explainer_instance function
  
  Args:
    list of texts represented as strings - prediction will be run against each 
  
  Returns:
    numpy array of tuples - each represents (p(is hyperpartisan),p(is not hyperpartisan))
  """
  
  probs = []
  if verbose:
    print("base text length =", len(in_text_list[0]))
    print("fm_predict_proba - number of elements in text list ", len(in_text_list))
  text_list = in_text_list
  prob1 = prob2 = 0 
  t0 = time.time()
#   call to hyperpartisan classifier
  results = production_hp.predict(text_list)
  t1 = time.time()

  if verbose:
    print("predict run time = ",str(datetime.timedelta(seconds=round(t1-t0,0))))
  for result in results :  
    if result['class'] == classes[0]:
      prob1 = result['score']
      prob2 = 1 - prob1
    else: 
      prob2 = result['score']
      prob1 = 1 - prob2
    probs.append([prob1,prob2])
    
  return(np.array(probs))

In [9]:
# quick test if needed
test_txt1 = []
test_txt1.append("President Donald Trump was in the Philippines on Monday as part of the final stop on a whirlwind, 12-day tour of Asia that included warm receptions by the gracious hosts of Japan, South Korea, China and Vietnam, according to Fox News.")
res = fm_predict_proba(test_txt1)
res

/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


array([[0.083, 0.917]])

# Fetch data, train a simple LR classifier

In [10]:
from sklearn import linear_model

# test_file = "/Users/rick/factmata/article quotes v2.csv"
test_file = "/Users/rick/factmata/article quotes - 2018-08-09.csv"

# training = pd.read_csv("/Users/rick/factmata/train.csv")
training = pd.read_csv("/Users/rick/factmata/factmata-quality-engine/factnlp/models/hyperpartisanship/current/train.csv")

testing = pd.read_csv(test_file)
class_names = ['Biased','Unbiased']

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(training['text'])
train_targets = training['tag']
test_vectors = vectorizer.transform(testing['text'])
test_targets = testing['tag']

/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [11]:
good_neutral_tags = ['agree','citation_needed','fair','funny','question']
bad_tags = ['one_sided',"unfair","wrong","nasty"]
biased_tags = ['one_sided']

## Basic logistic regression.

In [12]:
lr = linear_model.LogisticRegression(C=1e5)
lr.fit(train_vectors,train_targets)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [13]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, lr)

## Create an explainer object 
Select text and run explainer
Create list of explainer words 

In [14]:
# s = testing['id']


## Choose which model to run

In [15]:
LR_MODEL = 0
FM_MODEL = 1 

# model_used = LR_MODEL
model_used = FM_MODEL

if model_used == FM_MODEL : 
  pred_proba_func = fm_predict_proba
  num_samples = 1000
else :
  pred_proba_func = c.predict_proba
  num_samples = 5000

In [16]:
##  function to return all rows (i.e. annotations) matching a page_id
def get_quotes_list(testing, page_id):
  """
   Args:
     testing: panda data frame with text, quotes etcx
     page_id: id of piece of text (as represented in 'page__id' of data frame)
   
   Returns:
     List of all rows in data frame that match page_id (i.e. all rows which represent
     quotes from this piece of text)
  """

#   TODO improve error handling ?
#   if page_id not in testing['page__id']:
#     return [] 
  page_refs = (testing['page__id'] == page_id)
#   for df in testing[page_refs]:
#     print("page_id , quote tag ",df['page__id'] ,df['tag'])
#     if testing[page_ref]['tag'] in biased_tags:
#       pages_list.append[page_ref]
#     else:
#       print("get quotes - reject ", page_ref, testing[page_ref]['tag'])

  pages_list = testing[page_refs]

  print("pre-filter",page_id,pages_list['tag'])
  pages_list = pages_list[pages_list['tag'] == "one_sided"]
  print("post-filter",page_id,pages_list['tag'])
#   print("pages list = ", pages_list['tag'])

  return pages_list

def get_word_scores(word_list, text, text_as_dict, quotes_region): 
  """
  Produces score on how often the words appear in the text and in the quotes region
  (Simpler version of get_word_stats, which uses a list of quotes rather than unified
  quote regions)

  Args:
    words: a subset of words or ngrams from the text 
    text : the text itself
    text_as_dict:  dict with key as a word and value its occurence count e.g {'fall':3}
    quotes_region : list of tuples in form (start, stop) indicating position of quote regions in text

  Returns: Dict - key is word, value is dict with annt_count - no of occurences of word in quote region
                                                  text_count - no of occurences of word in entire text 
                                
  """
  
  debug_list = [(start,stop) for (start,stop) in quotes_region]
#   print("quotes region = ", sorted(quotes_region))
  all_quotes = ' '.join([text[start:stop] for (start,stop) in quotes_region])
  all_quotes_count_dict = Counter(np.array(re.split(r'%s|$' % r'\W+' , all_quotes)))
  scores = {word:{'annt_count':0,
                 'text_count':0,
                  'weight':weight
                } for (word,weight) in word_list}
  for word,_ in word_list:  
    scores[word]['text_count'] = text_as_dict[word]
    scores[word]['annt_count'] = all_quotes_count_dict[word]    
  return scores

def display_stats(exp_words, stats):
  for word in exp_words:
    if stats[word]['annt_count'] != 0 :
      word_stat = stats[word]
      idf = word_stat['idf']
    if stats[word]['annt_freq'] > stats[word]['text_freq'] :
      print("%20s >>> text-freq = %.5f  quote-freq = \x1b[31m%.5f\x1b[0m"%(word, stats[word]['text_freq'] , stats[word]['annt_freq']))
    else:
      print("%20s >>> text-freq = %.5f  quote-freq = %.5f"%(word, stats[word]['text_freq'] , stats[word]['annt_freq']))
      
  print('\n')
  

## Text and index utility functions - part 2 

In [17]:
import random

def build_quoted_regions(quoted_parts,randomize = False):
    """
    sort quoted parts, then merge any which overlap or form
    continuous quotes

    Args :
        list of tuples representing sections of text
    Returns:
        The same but with sections merged to continuous regions where
        appropriate
    """
    def purge_list(target_list, remove_list):
        for i in reversed(remove_list):
            del(target_list[i])
        return target_list

    remove_list = []
    ####  remove any dodgy tuples ####
    # check for tuples with start point after stop point, or negative start
    for i in range(len(quoted_parts)):
        start,stop = quoted_parts[i]
        if (start < 0) or (start > stop):
            print("warning - start point should be non-negative and not greater than stop point in ", quoted_parts[i])
            remove_list.append(i)
    # for i in reversed(remove_list):
    #     del(quoted_parts[i])
    purge_list(quoted_parts,remove_list)

    #### main sort and merge ####
    quoted_parts = sorted(quoted_parts)
    quoted_parts_len = len(quoted_parts)
    remove_list = []
    for i in range(quoted_parts_len - 1):
        # compare end of one part with start of other - merge if overlap
        start1, stop1 = quoted_parts[i]
        start2, stop2 = quoted_parts[i + 1]
        if stop1 >= start2:
            if stop2 <= stop1:
                # remove subsumed part
                remove_list.append(i+1)
            else:
                # second part absorbs first, first is removed
                quoted_parts[i + 1] = (start1, stop2)
                remove_list.append(i)
    purge_list(quoted_parts,remove_list)

    return quoted_parts
  
def randomize_regions(quoted_parts,text_len):
  """
  Takes some quoted regions and returns randomized control regions of 
  the same length and number
  """
  num_regions = len(quoted_parts)
  num_rand_regions = 0 
  rand_regions = []
  attempts = 0
  while (num_rand_regions < num_regions) and (attempts < 100):
    start = random.randint(0,text_len-1)
    (quote_start,quote_stop) = quoted_parts[num_rand_regions]
    region_len = quote_stop - quote_start 
    stop = start + region_len 
    print(start,stop)
    if  stop > text_len - 1 :
      attempts += 1 
      continue
    else:
      overlap = False
      for i in range(num_rand_regions):
        if (start in rand_regions[i]) or (stop in rand_regions[i]):
          overlap = True
          continue
#   if there's an overlap don't keep this region , find a new one 
      if overlap:
        continue
      else:
        rand_regions.append((start,stop))
        num_rand_regions += 1
  return rand_regions
        
##############################

def get_quote_idxs(txt, quotes, err_file = None):
    """
    Args:
        txt - string
        quotes - list of string
    Returns:
        list of tuples indicating (start,stop) position of each quote in txt (if doesn't appear it's ignored)
    """
    quoted_idxs = []
    for quote in quotes:
      start = txt.find(quote)
      if start != -1 :
        quoted_idxs.append((start,start+len(quote)))
      else:
        if err_file is not None:
          err_file.write("\n*START OF TEXT*\n")
          err_file.write(txt)
          err_file.write("\n")
          err_file.write("** END OF TEXT*\n")
          err_file.write("quote = ")
          err_file.write(quote)
          err_file.write("\n***\n")
        
    return quoted_idxs


In [18]:
# test code 

col_names =  ['quote']
quotes_df  = pd.DataFrame(columns = col_names)
# my_df
# If you want to add a record to the dataframe it would be better to use:

# quotes_df.loc[len(my_df)] = ["lazy dog"]
quotes_df.loc[0] = ["lazy dog"]
quotes_df.loc[1] = ["jumped over"]
# quotes_df.loc[1] = ["quick fox"]
print(quotes_df)

text = 'the quick brown fox jumped over the lazy dog more dog rhubarb rhubarb dog rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarbrhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb rhubarb'
text_dict = Counter(np.array(re.split(r'%s|$' % r'\W+' , text)))
print("text dictionary")
print(text_dict)

words_list = [("dog",0.4),("rhubarb",0.2)]

quotes_list = [quote for quote in quotes_df['quote']]
test_quote_idxs = get_quote_idxs(text,quotes_list)


quote_region = build_quoted_regions(test_quote_idxs)
print("quote region", quote_region)
my_score = get_word_scores(words_list, text, text_dict, quote_region)
print(my_score)


# display_stats(words,my_stats)

         quote
0     lazy dog
1  jumped over
text dictionary
Counter({'rhubarb': 52, 'dog': 3, 'the': 2, 'quick': 1, 'brown': 1, 'fox': 1, 'jumped': 1, 'over': 1, 'lazy': 1, 'more': 1, 'rhubarbrhubarb': 1})
quote region [(20, 31), (36, 44)]
{'dog': {'annt_count': 1, 'text_count': 3, 'weight': 0.4}, 'rhubarb': {'annt_count': 0, 'text_count': 52, 'weight': 0.2}}


/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [19]:
%load_ext autoreload
%autoreload 2
import lime

## Main Experiment 

In [20]:
# create sorted list of individual page__id's
page_id_set = set(testing['page__id'])
page_id_list = list(page_id_set)
page_id_list.sort()

In [21]:
# LIME parameters
num_features = 10

# experiment parameters
debug_display = False
MIN_TXT_SIZE = 500
rand_control_num = 500           # number of times to bootstrap random samples from text
num_samples = 5000                # number of samples LIME uses to build its model
# =====>
stop_words = True                # True - stop words left in ; False - stop words left out 

num_articles = len(page_id_list)

# for speed cache stop words
cachedStopWords = stopwords.words("english")

def open_results_files(ts):
  ts = time.localtime()
  results_file = "results/results_" + time.strftime("%H%M%S_%d%m%Y_withstops.json", ts)
  errors_file = "results/errors_" + time.strftime("%H%M%S_%d%m%Y.txt", ts)
  try:
    fr = open(results_file,'w')
  except:
    print("error opening file %s" % (results_file))
  try:
    fe = open(errors_file,'w')
  except:
    print("error opening file %s" % (errors_file)) 
  return fr, fe 

###################################################
## CREATE EXPLAINER OBJECT                       ##
###################################################
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(split_expression=r'\W+',class_names=class_names,ngram_size=1,stop_words=stop_words)

# def run_experiment(num_articles, num_features,num_samples)
# ********** MAIN LOOP *************
# look at first [num_articles] pages
all_word_scores = []
all_random_word_scores = []

ts = time.localtime()
timestamp =  time.strftime("%H%M%S_%d%m%Y", ts)

desc_string = "500 sets of random words, added word weight from LIME"
short_desc = "E200 current master version - unigrams, stop words removed; v2 of data"
num_trials = 1


#  note - JSON keys have to be strings (unlike python dict keys)
experiment_dict = {'timestamp':timestamp,
                   'description': desc_string ,
                   'short_desc' : short_desc,
                     'trials':
                       {str(trial):
                         {
                           'num_articles':num_articles,
                           'num_features':num_features,
                           'num_samples':num_samples,
                           'rand_sample_size':rand_control_num,
                           'stop_words': stop_words,
                           'corpus':test_file,
                           'run_time':'',
                           'articles':{}
                         }for trial in range(num_trials)
                       }
                  }
# experiment_dict = {'timestamp':timestamp,
#                    'description': desc_string ,
#                    'short_desc' : short_desc,
#                      'trials':
#                        {str(trial):
#                          {
#                            'num_articles':num_articles,
#                            'num_features':num_features,
#                            'num_samples':num_samples,
#                            'rand_sample_size':rand_control_num,
#                            'stop_words': stop_words,
#                            'corpus':test_file,
#                            'run_time':'',
#                            'articles':
#                              {str(page_id):
#                                {
#                                'score':0.0,
#                                'txt_len':0,
#                                'quote_len':0,
#                                'explainer_words':[],
#                                'random_words':[]
#                                } for page_id in page_id_list[:num_articles]                    
#                              }
#                          }for trial in range(num_trials)
#                        }
#                   }
                    

res_file, err_file = open_results_files(ts)


page_dict = {}
# how many times to select random word comparision set from same text 

trial_key = str(0)
# for page_id in page_id_list[:num_articles]:
# for page_id in page_id_list:

###################################################
## LOOP THROUGH ARTICLES                         ##
###################################################
for page_id in page_id_list:
# for page_id in page_id_list[:2]:
  page_key = str(page_id)
  print("page id = ",page_id)
#  get text of article
  text = testing[testing['page__id'] == page_id].iloc[0]['text']
  text = text.lower()

#   skip to next article, if article text is too small
  if len(text) < MIN_TXT_SIZE:
    continue

  #   Run LIME explainer to get explainer words
  exp = explainer.explain_instance(text, pred_proba_func, num_features=num_features, num_samples=num_samples)
  if debug_display:
    print('Probability(biased) =', c.predict_proba([text])[0,0])
  # if classifier giving bad result - skip this record
  if(exp.classifier_error == True):
    print("classifier error detected")
    continue
  # find all quotes for given article - if none matching criteria, skip this record
  quotes_list_df = get_quotes_list(testing, page_id)
  if(quotes_list_df.empty):
    print(">>>  list empty, skip <<<")
    continue
      
  experiment_dict['trials'][trial_key]['articles'][page_key]= {
                                                            
                                                        'score':0.0,
                                                        'txt_len':0,
                                                        'quote_len':0,
                                                        'explainer_words':[],
                                                        'random_words':[]
    
                                                            }
  experiment_dict['trials'][trial_key]['articles'][page_key]['score'] = exp.score
  exp_words = [word for (word,weight) in exp.as_list()]


  quotes_list = [quote.lower() for quote in quotes_list_df['quote']]
  quotes_idxs = get_quote_idxs(text, quotes_list, err_file)
  quote_regions = build_quoted_regions(quotes_idxs)
  text_as_list = np.array(re.split(r'%s|$' % r'\W+' , text))
  text_count_dict = Counter(text_as_list) 
                            
#   process text into a set of words, removing single character words and stop words (if required)
#   print("text_as_list = ", text_as_list)
  text_as_list = np.array(list(filter(lambda x: len(x)>1, text_as_list)))
  text_as_set = set(text_as_list)
  if stop_words == False:
    text_as_set = set(filter(lambda x: x not in cachedStopWords,text_as_set))  
    
  text_as_set_list  = np.array(list(text_as_set))
    
  word_scores = get_word_scores(exp.as_list(), text, text_count_dict, quote_regions )
  experiment_dict['trials'][trial_key]['articles'][page_key]['explainer_words'] = word_scores

  #   control group of words randomly selected from article    
  for i in range(rand_control_num):
    all_random_word_scores = []
    rand_choice = np.random.choice(text_as_set_list.size, num_features, replace=False)
    random_words = [(random,0.0) for random in text_as_set_list[rand_choice]]    
    if debug_display:
      print(random_words)
    random_word_scores = get_word_scores(random_words,text,text_count_dict, quote_regions)
    experiment_dict['trials'][trial_key]['articles'][page_key]['random_words'].append(random_word_scores)
    
json.dump(experiment_dict,res_file)
res_file.close()
err_file.close()



page id =  132


/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


{"level": "WARNING", "message": "An error has been found predicting Hyperpartisanship.", "log_timestamp": "2018-09-07 09:21:50.180316", "filename": "hyperpartisanship_predictor.py", "function": "filter_results", "path": "/Users/rick/factmata/factmata-quality-engine/factnlp/factnlp/hyperpartisanship/hyperpartisanship_predictor.py", "logger_name": "factnlp", "lineno": 85, "process": 46898, "thread": 140736053298048, "details": {"message": "An error has been found predicting Hyperpartisanship.", "stage": "bias_predict_error", "text": "t obama a s a nutritional s nutritional nutritional obama obama obama a obamas i called i s t t s...", "error": "Not understandable English."}}
{"level": "WARNING", "message": "An error has been found predicting Hyperpartisanship.", "log_timestamp": "2018-09-07 09:21:50.181223", "filename": "hyperpartisanship_predictor.py", "function": "filter_results", "path": "/Users/rick/factmata/factmata-quality-engine/factnlp/factnlp/hyperpartisanship/hyperpartisanship_

/Users/rick/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


pre-filter 132 2097    citation_needed
2098              nasty
2099              nasty
2100              wrong
2101              nasty
Name: tag, dtype: object
post-filter 132 Series([], Name: tag, dtype: object)
>>>  list empty, skip <<<
page id =  133
pre-filter 133 2095    citation_needed
2096             unfair
Name: tag, dtype: object
post-filter 133 Series([], Name: tag, dtype: object)
>>>  list empty, skip <<<
page id =  134
pre-filter 134 2136    nasty
Name: tag, dtype: object
post-filter 134 Series([], Name: tag, dtype: object)
>>>  list empty, skip <<<
page id =  135
pre-filter 135 2093             unfair
2094    citation_needed
Name: tag, dtype: object
post-filter 135 Series([], Name: tag, dtype: object)
>>>  list empty, skip <<<
page id =  136
{"level": "WARNING", "message": "An error has been found predicting Hyperpartisanship.", "log_timestamp": "2018-09-07 09:23:10.753841", "filename": "hyperpartisanship_predictor.py", "function": "filter_results", "path": "/Users/rick/f

pre-filter 145 2050              wrong
2051    citation_needed
2052    citation_needed
2053    citation_needed
Name: tag, dtype: object
post-filter 145 Series([], Name: tag, dtype: object)
>>>  list empty, skip <<<
page id =  146
pre-filter 146 2044    unfair
2045    unfair
2046     nasty
2047     nasty
2048     nasty
2049     wrong
Name: tag, dtype: object
post-filter 146 Series([], Name: tag, dtype: object)
>>>  list empty, skip <<<
page id =  147
pre-filter 147 2038     wrong
2039     wrong
2040     nasty
2041    unfair
2042    unfair
2043     nasty
Name: tag, dtype: object
post-filter 147 Series([], Name: tag, dtype: object)
>>>  list empty, skip <<<
page id =  148
pre-filter 148 2122    nasty
2123    nasty
2124    nasty
2125    nasty
2126    nasty
2127    nasty
2128    nasty
2129    nasty
Name: tag, dtype: object
post-filter 148 Series([], Name: tag, dtype: object)
>>>  list empty, skip <<<
page id =  149
pre-filter 149 2119              nasty
2120              nasty
2121    citat

KeyboardInterrupt: 

In [ ]:
res_file.name


In [ ]:
"he" in cachedStopWords